In [20]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import train_test_split

In [21]:
PATH_CSV_LABELS = '../tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../../Nuovi Dati/TemplateGeneratiClean'
PATH_MODELLO = '../../Modelli-v2/modello_pesi_inizializzati_black_stains'
PATH_MODELLO_FINALE_RIADDESTRATO = '../../Modelli-v2/modello_finale_riaddestrato_black_stains'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

HUBER_DELTA = 1.0

In [22]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,AgatielloRoberto_000,0.648876,50.428711
1,AgatielloRoberto_001,0.759503,61.300294
2,AgatielloRoberto_002,0.747500,60.120716
3,AgatielloRoberto_003,0.818899,67.137182
4,AgatielloRoberto_004,0.825895,67.824714


In [23]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=44)

In [25]:
print(f'X_train: {len(X_train)}')
print(f'X_test: {len(X_test)}')

X_train: 574
X_test: 144


In [26]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2080800)           0         
                                                                 
 dense (Dense)               (None, 64)                133171264 
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 133,171,649
Trainable params: 133,171,649
Non-trainable params: 0
__________________________________________

In [27]:
# model.fit(x=X_train, y=y_train, batch_size=1, epochs=20, validation_data=(X_cv, y_cv))

In [28]:
#model.predict(X)

In [29]:
# model.save(PATH_MODELLO_FINALE)

## Addestramento nuovo modello con Huber loss

In [30]:
conv1_weights = model.layers[0].weights
dense1_weights = model.layers[3].weights
dense2_weights = model.layers[4].weights

new_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS), weights=conv1_weights),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', weights=dense1_weights),
    layers.Dense(1, weights=dense2_weights)
])

In [31]:
new_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=HUBER_DELTA), metrics=['mae'])

In [32]:
new_model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/10


2023-07-26 12:51:10.438060: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


574/574 [==============================] - 62s 107ms/step - loss: 17.0177 - mae: 17.5117
Epoch 2/10
574/574 [==============================] - 62s 107ms/step - loss: 16.0748 - mae: 16.5667
Epoch 3/10
574/574 [==============================] - 62s 107ms/step - loss: 15.0925 - mae: 15.5841
Epoch 4/10
574/574 [==============================] - 62s 108ms/step - loss: 14.5142 - mae: 15.0051
Epoch 5/10
574/574 [==============================] - 61s 107ms/step - loss: 11.7404 - mae: 12.2297
Epoch 6/10
574/574 [==============================] - 62s 107ms/step - loss: 9.2823 - mae: 9.7690
Epoch 7/10
574/574 [==============================] - 62s 108ms/step - loss: 7.4736 - mae: 7.9545
Epoch 8/10
574/574 [==============================] - 62s 108ms/step - loss: 5.7760 - mae: 6.2564
Epoch 9/10
574/574 [==============================] - 61s 107ms/step - loss: 4.5915 - mae: 5.0734
Epoch 10/10
574/574 [==============================] - 62s 108ms/step - loss: 4.1624 - mae: 4.6357


In [33]:
#new_model.predict(X)

In [34]:
#y

In [35]:
new_model.save(PATH_MODELLO_FINALE_RIADDESTRATO)

INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_black_stains/assets


INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_black_stains/assets


### Import nuovo modello

In [4]:
new_model = tf.keras.models.load_model(PATH_MODELLO_FINALE_RIADDESTRATO)

Metal device set to: Apple M1 Pro


2023-07-13 10:26:56.552384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-13 10:26:56.552752: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
def avg(lst):
    return sum(lst) / len(lst)

In [19]:
avg(abs(new_model.predict(X) - y))

23/23 [==============================] - 3s 133ms/step


array([5.80999414])